# Question 6.1(a)

### We can see that as we increase the value of d we see that our n_full / n_avg does tend to become 2 as what we would expected

In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from itertools import product
import numpy as np

def remove_rows(X, y, X_original=None,y_original=None):
    if X_original is None and y_original is None:
        X_original, y_original = X.copy(), y.copy()
    if X.shape[0] <= 1:
        return X, y
    for i in range(X.shape[0]):
        X_temp = np.delete(X, i, axis=0)
        y_temp = np.delete(y, i, axis=0)
        knn = KNeighborsClassifier(n_neighbors=1)
        knn.fit(X_temp, y_temp)
        y_pred = knn.predict(X_original)
        if accuracy_score(y_original, y_pred) == 1:
            X = np.delete(X, i, axis=0)
            y = np.delete(y, i, axis=0)
            return remove_rows(X, y)
    return X, y

In [5]:
d = [2, 4, 6]
for D in d:
    final_y_sizes = []
    num_iteration = 8*D
    for _ in range(num_iteration):
        X = np.array(list(product([0, 1], repeat=D)))
        y = np.random.randint(2, size=(X.shape[0], 1)).ravel()
        X, y = remove_rows(X, y)
        final_y_sizes.append(y.shape[0])
    average_y_size = np.mean(final_y_sizes)
    print(f"d={D}: n_full={2**D}, Avg. req. points for memorization n_avg={average_y_size:.2f}, n_full/n_avg={(2**D)/average_y_size}")

d=2: n_full=4, Avg. req. points for memorization n_avg=2.56, n_full/n_avg=1.5609756097560976
d=4: n_full=16, Avg. req. points for memorization n_avg=8.81, n_full/n_avg=1.8156028368794326
d=6: n_full=64, Avg. req. points for memorization n_avg=33.00, n_full/n_avg=1.9393939393939394


# Question 6.1(b)

### We used that the class is 3 and we would expected that the ratio of n_full to n_avg will tend to 3/(3-1) = 1.5 and as we can see it does tend to 1.5 as expected

In [7]:
d = [2, 4, 6]
for D in d:
    final_y_sizes = []
    num_iteration = 8*D
    for _ in range(num_iteration):
        X = np.array(list(product([0, 1], repeat=D)))
        y = np.random.randint(3, size=(X.shape[0], 1)).ravel()
        X, y = remove_rows(X, y)
        final_y_sizes.append(y.shape[0])
    average_y_size = np.mean(final_y_sizes)
    print(f"d={D}: n_full={2**D}, Avg. req. points for memorization n_avg={average_y_size:.2f}, n_full/n_avg={(2**D)/average_y_size}")

d=2: n_full=4, Avg. req. points for memorization n_avg=3.38, n_full/n_avg=1.1851851851851851
d=4: n_full=16, Avg. req. points for memorization n_avg=11.56, n_full/n_avg=1.3837837837837839
d=6: n_full=64, Avg. req. points for memorization n_avg=43.31, n_full/n_avg=1.4776334776334776


### We used that the class is 6 and we would expected that the ratio of n_full to n_avg will tend to 6/(6-1) = 1.2 and as we can see it does tend to 1.2 as expected

In [10]:
d = [2, 4, 6]
for D in d:
    final_y_sizes = []
    num_iteration = 8*D
    for _ in range(num_iteration):
        X = np.array(list(product([0, 1], repeat=D)))
        y = np.random.randint(6, size=(X.shape[0], 1)).ravel()
        X, y = remove_rows(X, y)
        final_y_sizes.append(y.shape[0])
    average_y_size = np.mean(final_y_sizes)
    print(f"d={D}: n_full={2**D}, Avg. req. points for memorization n_avg={average_y_size:.2f}, n_full/n_avg={(2**D)/average_y_size}")

d=2: n_full=4, Avg. req. points for memorization n_avg=3.62, n_full/n_avg=1.103448275862069
d=4: n_full=16, Avg. req. points for memorization n_avg=13.75, n_full/n_avg=1.1636363636363636
d=6: n_full=64, Avg. req. points for memorization n_avg=53.17, n_full/n_avg=1.2037617554858935


# Question 6.2(a)

### We have that we applied algorithm 8 into the breast cancer dataset which is binary classification. We see that the number of clauses are 59 and the accuracy is about 59.2. Then I tried to improve the algorithm by instead of using the sum but use gradient descent to compute the weight in which we want to combine our input and we can see that the number of clauses are 49 and the accuracy is 66.23

In [7]:
import math
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [9]:
def memorize(data, labels):
    thresholds = 0
    table = []
    for row, x in enumerate(data):
        #print(x)
        sum_features = sum(x)
        table.append((sum_features, labels[row]))
    sorted_table = sorted(table, key=lambda x: x[0])
    class_value = 0
    for row in sorted_table:
        if row[1] != class_value:
            class_value = row[1]
            thresholds += 1
    minthreshs = math.log2(thresholds + 1)
    d = data.shape[1]
    mec = (minthreshs * (d + 1)) + (minthreshs + 1)
    return mec, sorted_table

def generate_if_then_clauses(data, labels):
    mec, sorted_table = memorize(data, labels)
    clauses = []
    previous_class_value = sorted_table[0][1]
    for i in range(1, len(sorted_table)):
        current_class_value = sorted_table[i][1]
        if current_class_value != previous_class_value:
            threshold = sorted_table[i][0]
            clause = f"if sum_features < {threshold} then class_value = {previous_class_value} else class_value = {current_class_value}"
            clauses.append(clause)
            previous_class_value = current_class_value
    return clauses
def apply_clauses(clauses, features):
    sum_features = sum(features)
    for clause in clauses:
        threshold = float(clause.split('<')[1].split('then')[0].strip())
        class_values = [int(x.split('=')[1].strip()) for x in clause.split('then')[1].split('else')]
        if sum_features < threshold:
            return class_values[0]
        else:
            return class_values[1]


In [10]:
def memorize_2(data, labels, weights):
    thresholds = 0
    table = []
    for row, x in enumerate(data):
        weighted_sum = np.dot(weights, x)
        table.append((weighted_sum, labels[row]))
    sorted_table = sorted(table, key=lambda x: x[0])
    class_value = 0
    for row in sorted_table:
        if row[1] != class_value:
            class_value = row[1]
            thresholds += 1

    min_threshs = math.log2(thresholds + 1)
    d = len(weights)  
    mec = (min_threshs * (d + 1)) + (min_threshs + 1)

    return mec, sorted_table

def generate_if_then_clauses_2(data, labels, weights):
    mec, sorted_table = memorize_2(data, labels, weights)
    clauses = []
    previous_class_value = sorted_table[0][1]
    for i in range(1, len(sorted_table)):
        current_class_value = sorted_table[i][1]
        if current_class_value != previous_class_value:
            threshold = sorted_table[i][0]
            clause = f"if sum_features < {threshold} then class_value = {previous_class_value} else class_value = {current_class_value}"
            clauses.append(clause)
            previous_class_value = current_class_value
    return clauses

def apply_clauses_2(clauses, features, weights):
    weighted_sum = np.dot(weights, features)
    for clause in clauses:
        threshold = float(clause.split('<')[1].split('then')[0].strip())
        class_values = [int(x.split('=')[1].strip()) for x in clause.split('then')[1].split('else')]
        if weighted_sum < threshold:
            return class_values[0]
        else:
            return class_values[1]

def sigmoid(scores):
    return 1 / (1 + np.exp(-scores))

def calculate_loss(predicted_probs, true_labels):
    return -np.mean(true_labels * np.log(predicted_probs) + (1 - true_labels) * np.log(1 - predicted_probs))

def calculate_gradient(features, weights, true_labels):
    predicted_probs = sigmoid(np.dot(features, weights))
    return np.dot(features.T, (predicted_probs - true_labels)) / len(true_labels)

def update_weights(weights, gradient, learning_rate):
    return weights - learning_rate * gradient

In [186]:
from sklearn.preprocessing import StandardScaler
breast_cancer = datasets.load_breast_cancer()
features = breast_cancer.data
labels = breast_cancer.target
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.4, random_state=39)
scaler = StandardScaler()
features_train = scaler.fit_transform(features_train)
features_test = scaler.transform(features_test)
clauses = generate_if_then_clauses(features_train, labels_train)
predicted_labels = [apply_clauses(clauses, features) for features in features_test]
accuracy = np.mean(predicted_labels == labels_test)
#print(clauses)
print(f"Number of if then else clauses : {len(clauses)}")
print(f'Accuracy: {accuracy * 100}%')

Number of if then else clauses : 59
Accuracy: 59.210526315789465%


In [242]:
breast_cancer = datasets.load_breast_cancer()
features = breast_cancer.data
labels = breast_cancer.target
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.4, random_state=39)
scaler = StandardScaler()
features_train = scaler.fit_transform(features_train)
features_test = scaler.transform(features_test)
num_features = features_train.shape[1]
weights = np.random.rand(num_features)  
learning_rate = 1e-4
num_iterations = 100
for i in range(num_iterations):
    predicted_probs = sigmoid(np.dot(features_train, weights))
    loss = calculate_loss(predicted_probs, labels_train)
    print(f"Iteration {i}: Loss = {loss}")
    gradient = calculate_gradient(features_train, weights, labels_train)
    weights = update_weights(weights, gradient, learning_rate)
mec, sorted_table = memorize_2(features_train, labels_train,weights)
#print(f"Mec : {mec}")
clauses = generate_if_then_clauses_2(features_train, labels_train,weights)
predicted_labels = [apply_clauses_2(clauses, features,weights) for features in features_test]
accuracy = np.mean(predicted_labels == labels_test)
print(f"Number of if then else clauses : {len(clauses)}")
print(f'Accuracy: {accuracy * 100}%')

Iteration 0: Loss = 7.163664870403102
Iteration 1: Loss = 7.162862606308592
Iteration 2: Loss = 7.162073878720052
Iteration 3: Loss = 7.161271766249558
Iteration 4: Loss = 7.160471253975668
Iteration 5: Loss = 7.159669211046136
Iteration 6: Loss = 7.158868954969445
Iteration 7: Loss = 7.158067183455754
Iteration 8: Loss = 7.1572654723063245
Iteration 9: Loss = 7.156465334938003
Iteration 10: Loss = 7.155663765756041
Iteration 11: Loss = 7.154862253461406
Iteration 12: Loss = 7.154073820642462
Iteration 13: Loss = 7.153272457571218
Iteration 14: Loss = 7.152471103826277
Iteration 15: Loss = 7.151671493674197
Iteration 16: Loss = 7.150870329006266
Iteration 17: Loss = 7.15006922644442
Iteration 18: Loss = 7.149268322466106
Iteration 19: Loss = 7.148467290354786
Iteration 20: Loss = 7.147666460342999
Iteration 21: Loss = 7.146649936878199
Iteration 22: Loss = 7.145849288679222
Iteration 23: Loss = 7.145048578461271
Iteration 24: Loss = 7.1442479971016395
Iteration 25: Loss = 7.14344754446

# Question 6.2(b)

### We applied our method and modified method to two different datasets including the iris dataset and also the AND gate and we see that for both of the dataset the clause stay the same but that is already minimum which is 1 but we get higher accuracy for the modified method but for AND gate the accuracy stay the same which make sense.

In [11]:
iris = datasets.load_iris()
features = iris.data
labels = iris.target
mask = labels < 2
features = features[mask]
labels = labels[mask]
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.4, random_state=39)
clauses = generate_if_then_clauses(features_train, labels_train)
predicted_labels = [apply_clauses(clauses, features) for features in features_test]
accuracy = np.mean(predicted_labels == labels_test)
#print(clauses)
print(f"Number of if then else clauses : {len(clauses)}")
print(f'Accuracy: {accuracy * 100}%')

Number of if then else clauses : 1
Accuracy: 90.0%


In [175]:
iris = datasets.load_iris()
features = iris.data
labels = iris.target
mask = labels < 2
features = features[mask]
labels = labels[mask]
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.4, random_state=39)
num_features = features_train.shape[1]
weights = np.random.rand(num_features)  
learning_rate = 1e-3
num_iterations = 100
for i in range(num_iterations):
    predicted_probs = sigmoid(np.dot(features_train, weights))
    loss = calculate_loss(predicted_probs, labels_train)
    print(f"Iteration {i}: Loss = {loss}")
    gradient = calculate_gradient(features_train, weights, labels_train)
    weights = update_weights(weights, gradient, learning_rate)
mec, sorted_table = memorize_2(features_train, labels_train,weights)
#print(f"Mec : {mec}")
clauses = generate_if_then_clauses_2(features_train, labels_train,weights)
predicted_labels = [apply_clauses_2(clauses, features,weights) for features in features_test]
accuracy = np.mean(predicted_labels == labels_test)
print(f"Number of if then else clauses : {len(clauses)}")
print(f'Accuracy: {accuracy * 100}%')

Iteration 0: Loss = 2.4406378853925714
Iteration 1: Loss = 2.4298102295240285
Iteration 2: Loss = 2.418987449990667
Iteration 3: Loss = 2.408169642717957
Iteration 4: Loss = 2.397356905419533
Iteration 5: Loss = 2.3865493376262497
Iteration 6: Loss = 2.3757470407155235
Iteration 7: Loss = 2.36495011794093
Iteration 8: Loss = 2.354158674462122
Iteration 9: Loss = 2.343372817374984
Iteration 10: Loss = 2.3325926557420678
Iteration 11: Loss = 2.3218183006232698
Iteration 12: Loss = 2.3110498651067783
Iteration 13: Loss = 2.300287464340239
Iteration 14: Loss = 2.289531215562164
Iteration 15: Loss = 2.278781238133546
Iteration 16: Loss = 2.268037653569707
Iteration 17: Loss = 2.2573005855723114
Iteration 18: Loss = 2.246570160061597
Iteration 19: Loss = 2.235846505208753
Iteration 20: Loss = 2.2251297514684842
Iteration 21: Loss = 2.2144200316116858
Iteration 22: Loss = 2.2037174807582844
Iteration 23: Loss = 2.1930222364101715
Iteration 24: Loss = 2.182334438484234
Iteration 25: Loss = 2.1

In [180]:
features = np.array([[0, 0],
                     [0, 1],
                     [1, 0],
                     [1, 1]])
binary_labels = np.array([0, 0, 0, 1])
features_train, features_test, labels_train, labels_test = train_test_split(features, binary_labels, test_size=0.1, random_state=39)

In [181]:
mec, sorted_table = memorize(features_train, labels_train)
print(f"Mec : {mec}")
clauses = generate_if_then_clauses(features_train, labels_train)
predicted_labels = [apply_clauses(clauses, features) for features in features_test]
accuracy = np.mean(predicted_labels == labels_test)
print(f"Number of if then else clauses : {len(clauses)}")
print(f'Accuracy: {accuracy * 100}%')

Mec : 5.0
Number of if then else clauses : 1
Accuracy: 100.0%


In [182]:
num_features = features.shape[1]
weights = np.random.rand(num_features)  
learning_rate = 1e-3
num_iterations = 1000
for i in range(num_iterations):
    predicted_probs = sigmoid(np.dot(features_train, weights))
    loss = calculate_loss(predicted_probs, labels_train)
    print(f"Iteration {i}: Loss = {loss}")
    gradient = calculate_gradient(features_train, weights, labels_train)
    weights = update_weights(weights, gradient, learning_rate)
mec, sorted_table = memorize_2(features_train, labels_train,weights)
print(f"Mec : {mec}")
clauses = generate_if_then_clauses_2(features_train, labels_train,weights)
predicted_labels = [apply_clauses_2(clauses, features,weights) for features in features_test]
accuracy = np.mean(predicted_labels == labels_test)
#print(clauses)
print(f"Number of if then else clauses : {len(clauses)}")
print(f'Accuracy: {accuracy * 100}%')

Iteration 0: Loss = 0.7860153645074522
Iteration 1: Loss = 0.7859753478161643
Iteration 2: Loss = 0.7859353463701076
Iteration 3: Loss = 0.7858953601651404
Iteration 4: Loss = 0.7858553891971191
Iteration 5: Loss = 0.7858154334619017
Iteration 6: Loss = 0.7857754929553451
Iteration 7: Loss = 0.7857355676733064
Iteration 8: Loss = 0.785695657611643
Iteration 9: Loss = 0.7856557627662112
Iteration 10: Loss = 0.785615883132868
Iteration 11: Loss = 0.7855760187074704
Iteration 12: Loss = 0.7855361694858743
Iteration 13: Loss = 0.7854963354639373
Iteration 14: Loss = 0.7854565166375148
Iteration 15: Loss = 0.7854167130024637
Iteration 16: Loss = 0.7853769245546399
Iteration 17: Loss = 0.7853371512898998
Iteration 18: Loss = 0.7852973932040991
Iteration 19: Loss = 0.7852576502930941
Iteration 20: Loss = 0.7852179225527406
Iteration 21: Loss = 0.7851782099788941
Iteration 22: Loss = 0.785138512567411
Iteration 23: Loss = 0.785098830314146
Iteration 24: Loss = 0.7850591632149552
Iteration 25: 

# Question 6.2(c)

### I think this make sense since we random our feature and labels which mean that there is no actual relationship between them which therefore mean that during testing the model is just guessing the labels and that why we see that the accuracy is always about 50%

In [245]:
features = np.random.rand(20000, 5)
labels = np.random.randint(2, size=20000)
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.5, random_state=39)
clauses = generate_if_then_clauses(features_train, labels_train)
print(f'Number of if-then clauses: {len(clauses)}')
predicted_labels = [apply_clauses(clauses, features) for features in features_test]
accuracy = np.mean(predicted_labels == labels_test)
print(f'Accuracy: {accuracy * 100}%')

Number of if-then clauses: 4988
Accuracy: 49.38%


In [246]:
features = np.random.rand(20000, 5)
labels = np.random.randint(2, size=20000)
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.5, random_state=39)
num_features = features.shape[1]
weights = np.random.rand(num_features)  
learning_rate = 1e-3
num_iterations = 1000
for i in range(num_iterations):
    predicted_probs = sigmoid(np.dot(features_train, weights))
    loss = calculate_loss(predicted_probs, labels_train)
    print(f"Iteration {i}: Loss = {loss}")
    gradient = calculate_gradient(features_train, weights, labels_train)
    weights = update_weights(weights, gradient, learning_rate)
clauses = generate_if_then_clauses(features_train, labels_train)
print(f'Number of if-then clauses: {len(clauses)}')
predicted_labels = [apply_clauses(clauses, features) for features in features_test]
accuracy = np.mean(predicted_labels == labels_test)
print(f'Accuracy: {accuracy * 100}%')

Iteration 0: Loss = 0.9640472754590846
Iteration 1: Loss = 0.9639170576525731
Iteration 2: Loss = 0.9637868855143359
Iteration 3: Loss = 0.9636567590415352
Iteration 4: Loss = 0.9635266782313243
Iteration 5: Loss = 0.9633966430808479
Iteration 6: Loss = 0.9632666535872422
Iteration 7: Loss = 0.963136709747635
Iteration 8: Loss = 0.9630068115591456
Iteration 9: Loss = 0.9628769590188847
Iteration 10: Loss = 0.9627471521239542
Iteration 11: Loss = 0.9626173908714485
Iteration 12: Loss = 0.9624876752584524
Iteration 13: Loss = 0.9623580052820425
Iteration 14: Loss = 0.9622283809392873
Iteration 15: Loss = 0.9620988022272468
Iteration 16: Loss = 0.9619692691429719
Iteration 17: Loss = 0.9618397816835051
Iteration 18: Loss = 0.9617103398458811
Iteration 19: Loss = 0.9615809436271257
Iteration 20: Loss = 0.961451593024256
Iteration 21: Loss = 0.9613222880342808
Iteration 22: Loss = 0.9611930286542006
Iteration 23: Loss = 0.961063814881007
Iteration 24: Loss = 0.9609346467116837
Iteration 25:

# Question 6.3(a)

In [1]:
import random
import string 
import zlib
import pandas as pd
import sys

In [3]:
def generating_random_string(length_of_string):
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length_of_string))
def compressing_string(string_to_compress):
    return zlib.compress(string_to_compress.encode())
def compressing_ratio(compressed_string, original_string):
    return len(compressed_string)/len(original_string)
input_lengths = [2,4,6,8,10, 20, 30, 40, 50,60,70,80,90,100,150,200,1000,10000,20000,30000]
compression_ratios = []
for length in input_lengths:
    input_string = generating_random_string(length)
    compressed_string = compressing_string(input_string)
    ratio = compressing_ratio(compressed_string, input_string)
    compression_ratios.append(ratio)
df = pd.DataFrame({'Input Length': input_lengths, 'Compression Ratio': compression_ratios})
print(df)


    Input Length  Compression Ratio
0              2           5.000000
1              4           3.000000
2              6           2.333333
3              8           2.000000
4             10           1.800000
5             20           1.400000
6             30           1.266667
7             40           1.200000
8             50           1.160000
9             60           1.133333
10            70           1.114286
11            80           1.100000
12            90           1.088889
13           100           1.070000
14           150           0.973333
15           200           0.910000
16          1000           0.779000
17         10000           0.753000
18         20000           0.752800
19         30000           0.751800


## Question 6.3(b)



We have that as you increase the size of the length it seems like the compression ratio started to decrease. This is because we have that for a very small length of strings for example 2,4,6 there is a lot of overhead of the compression algorithm itself which therefore result to a compressed string that is larger than the original string which therefore mean that the compression ratio is more than 1. As the length of the string increases the compression algorithm has more data to work on which they can find more patterns which can lead to a better compression ratio. However, in the case of a long random string composed of ASCII letters and digits the random data has a very high entropy and high entropy data does not compressed effectively. Additionally, with the randomness there are not many repeating patterns which makes the data very hard to compress. This is why even for larger string the compression ratio does not decrease significantly and remains close to 1. Also as the string increase there should be the some pattern repeat so that why the ratio become 0.75 as well 